## Lesson-01 Assignment

#### 今天是2019年9月28日，今天世界上又多了一名AI工程师 :) 

`各位同学大家好，欢迎各位开始学习我们的人工智能课程。这门课程假设大家不具备机器学习和人工智能的知识，但是希望大家具备初级的Python编程能力。根据往期同学的实际反馈，我们课程的完结之后 能力能够超过80%的计算机人工智能/深度学习方向的硕士生的能力。`

## 本次作业的内容

#### 1. 复现课堂代码

在本部分，你需要参照我们给大家的GitHub地址里边的课堂代码，结合课堂内容，复现内容。

In [1]:
#最原始的rule base
simple_grammar= """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => Adj | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>   蓝色的 |  好看的 | 小小的"""

###语言规则也非常重要，错误输入会导致得不到想要的结果！！！


In [2]:
#抽取规则
def create_grammar(grammar_str, split_mark='=>', line_split_mark='\n', word_split_mark='|'):
    grammar={}
    for line in grammar_str.split(line_split_mark):
        if not line.strip(): 
            continue#过滤空行
        else:
            expr,stmt=line.split(split_mark)
            grammar[expr.strip()]=[s.split() for s in stmt.split(word_split_mark)]
            #这里是两个split，第一个是把字符串或列表外面再套列表，第二个才是切分带“|”的并且外面套个列表
    return grammar#要抽取规则放入字典

In [3]:
#注意部分有两层列表结构
create_grammar(simple_grammar)

{'sentence': [['noun_phrase', 'verb_phrase']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'Adj*': [['Adj'], ['Adj', 'Adj*']],
 'verb_phrase': [['verb', 'noun_phrase']],
 'Article': [['一个'], ['这个']],
 'noun': [['女人'], ['篮球'], ['桌子'], ['小猫']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见']],
 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [4]:
#生成句子
import random
choice=random.choice
def generate(gram,target):
    if target in gram.keys():#从grammar[target]中随机选一个拓展，并且把每一个继续拓展和拼接
        expanded=[generate(gram,t) for t in choice(gram[target])]
        return ''.join([e if e!='\n' else '\n' for e in expanded if e!='null'])
    
    else:#如果不能再继续拓展
        return target


In [5]:
generate(create_grammar(simple_grammar), target='sentence')

'一个小小的篮球看着这个小小的篮球'

In [6]:
corpus = './article_9k.txt'
FILE= open(corpus,encoding='utf-8').read()
def generate_by_pro(text_corpus, length=20):
    return ''.join(random.sample(text_corpus, length))

In [7]:
len(FILE)

33425826

In [8]:
print(FILE[50:])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
import jieba

max_length = 1000000 

sub_file = FILE[:max_length]

def cut(string):
    return list(jieba.cut(string))



In [10]:
TOKENS = cut(sub_file)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Jehuty\AppData\Local\Temp\jieba.cache
Loading model cost 1.582 seconds.
Prefix dict has been built succesfully.


In [11]:
len(TOKENS)

532454

In [12]:
from collections import Counter
%matplotlib inline

In [13]:
words_count = Counter(TOKENS)

In [14]:
words_count.most_common(20)

[('的', 28764),
 ('在', 7563),
 ('了', 6529),
 ('是', 4883),
 ('和', 4245),
 ('也', 2531),
 ('月', 2433),
 ('有', 2374),
 ('将', 2114),
 ('他', 1960),
 ('年', 1959),
 ('对', 1795),
 ('都', 1720),
 ('中', 1699),
 ('为', 1686),
 ('日', 1674),
 ('等', 1655),
 ('与', 1601),
 ('中国', 1597),
 ('上', 1583)]

In [15]:
words_with_fre = [f for w, f in words_count.most_common()]
words_with_fre[:10]

[28764, 7563, 6529, 4883, 4245, 2531, 2433, 2374, 2114, 1960]

In [16]:
_2_gram_words = [
    TOKENS[i] + TOKENS[i+1] for i in range(len(TOKENS)-1)
]

In [17]:
_2_gram_words[:10]


['此外自', '自本周', '本周6', '6月', '月12', '12日起', '日起除', '除小米', '小米手机', '手机6']

In [18]:
_2_gram_word_counts = Counter(_2_gram_words)

In [19]:
def get_1_gram_count(word):
    if word in words_count: return words_count[word]
    else:
        return words_count.most_common()[-1][-1]
    
def get_2_gram_count(word):
    if word in _2_gram_word_counts: return _2_gram_word_counts[word]
    else:
        return _2_gram_word_counts.most_common()[-1][-1]
    
def get_gram_count(word, wc):
    if word in wc: return wc[word]
    else:
        return wc.most_common()[-1][-1]

In [20]:
get_gram_count('XXX',words_count)

1

In [21]:
get_gram_count('XXX', _2_gram_word_counts)

1

In [22]:
def two_gram_model(sentence):
    # 2-gram langauge model
    tokens = cut(sentence)
    
    probability = 1
    
    for i in range(len(tokens)-1):
        word = tokens[i]
        next_word = tokens[i+1]
        
        _two_gram_c = get_gram_count(word+next_word, _2_gram_word_counts)
        _one_gram_c = get_gram_count(next_word, words_count)
        pro =  _two_gram_c / _one_gram_c
        
        probability *= pro
    
    return probability  

In [23]:
two_gram_model('此外自本周6月12日起除小米手机6等15款机型')


3.0550026391456175e-26

In [24]:
two_gram_model('前天早上吃晚饭的时候')

7.429922323539344e-07

#### 2. 请回答以下问题

回答以下问题，并将问题发送至 minchuian.gao@gmail.com 中：
```
    2.1. what do you want to acquire in this course？
    2.2. what problems do you want to solve？
    2.3. what’s the advantages you have to finish you goal?
    2.4. what’s the disadvantages you need to overcome to finish you goal?
    2.5. How will you plan to study in this course period?
```

#### 3. 如何提交
代码 + 此 jupyter 相关，提交至自己的 github 中(**所以请务必把GitHub按照班主任要求录入在Trello中**)；
第2问，请提交至minchuian.gao@gmail.com邮箱。
#### 4. 作业截止时间
此次作业截止时间为 2019.10.8日

#### 5. 完成以下问答和编程练习

>

## 基础理论部分

#### 0. Can you come up out 3 sceneraies which use AI methods? 

Ans: 机器翻译，自动问答客服，智能机器人

#### 1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: 用github下载作业任务，查找代码参考，上传作业。Jupyter用于做演示，多样的文字图片编辑适合做作业和提交，Pycharm用于大型项目调试以及细节debug，Crtl+鼠标左键快捷方便直接查看方法的使用

#### 2. What's the Probability Model?

Ans:概率模型就是统计单词出现的频次，得到各个单词出现概率的模型

#### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans:机器翻译，智能客服，语音识别

#### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans:因为基于语法（句子结构分析和模式匹配）的NLP效果很差，不可能找到一个适合所有情况的语法树，而只要一点改动就无法匹配语法树，而单纯用概率计算的效果更好，而且在数据变化的情况下，不用大幅改动程序。

#### 5. What's the Language Model;

Ans：语言模型通过data driven的方式，计算每个词，每句话出现的概率模型。

#### 6. Can you came up with some sceneraies at which we could use Language Model?


Ans:机器翻译，智能客服

#### 7. What's the 1-gram language model;

Ans:只考虑单词自己出现概率，不考虑上下文关联的语言概率模型

#### 8. What's the disadvantages and advantages of 1-gram language model;

Ans:优势是计算量小，劣势是准确度不够高

#### 9. What't the 2-gram models;

Ans:假定每个单词出现的概率受到前面一个单词影响的语言概率模型

## 编程实践部分

#### 1. 设计你自己的句子生成器

如何生成句子是一个很经典的问题，从1940s开始，图灵提出机器智能的时候，就使用的是人类能不能流畅和计算机进行对话。和计算机对话的一个前提是，计算机能够生成语言。

计算机如何能生成语言是一个经典但是又很复杂的问题。 我们课程上为大家介绍的是一种基于规则（Rule Based）的生成方法。该方法虽然提出的时间早，但是现在依然在很多地方能够大显身手。值得说明的是，现在很多很实用的算法，都是很久之前提出的，例如，二分查找提出与1940s, Dijstra算法提出于1960s 等等。

在著名的电视剧，电影《西部世界》中，这些机器人们语言生成的方法就是使用的SyntaxTree生成语言的方法。

> 
>

![WstWorld](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1569578233461&di=4adfa7597fb380e7cc0e67190bbd7605&imgtype=0&src=http%3A%2F%2Fs1.sinaimg.cn%2Flarge%2F006eYYfyzy76cmpG3Yb1f)

> 
>

在这一部分，需要各位同学首先定义自己的语言。 大家可以先想一个应用场景，然后在这个场景下，定义语法。例如：

在西部世界里，一个”人类“的语言可以定义为：
``` 
human = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 看看 | 找找 | 想找点
活动 = 乐子 | 玩的
"""
```

一个“接待员”的语言可以定义为
```
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

```




请定义你自己的语法: 

第一个语法：

In [25]:
you_need_replace_this_with_name_you_given = '''
sentence=suj adv v obj
suj=我|俺|我自己|大家一起
adv=tm|他妈|一定|必须
v=吹|吹爆|吹捧
obj=巨人|进击的巨人

'''

第二个语法：

In [26]:
you_need_replace_this_with_name_you_given_2 = '''
sentence=suj adv v adj* obj 语气*
adj*=null|adj* adj
suj=谁|哪个|你|你们
adv=敢不|会不|有胆不|竟然不
v=吹|吹爆|吹捧
adj=牛逼的|无敌的|宇宙第一的
obj=巨人|进击的巨人
语气*=null|语气* 语气
语气=哈哈|哇
'''

TODO: 然后，使用自己之前定义的generate函数，使用此函数生成句子。

In [27]:
generate(create_grammar(you_need_replace_this_with_name_you_given,split_mark='=', line_split_mark='\n', word_split_mark='|'),'sentence')


'俺tm吹爆巨人'

In [28]:
generate(create_grammar(you_need_replace_this_with_name_you_given_2,split_mark='=', line_split_mark='\n', word_split_mark='|'),'sentence')


'你们有胆不吹捧宇宙第一的宇宙第一的进击的巨人'

TODO: 然后，定义一个函数，generate_n，将generate扩展，使其能够生成n个句子:

In [29]:
# def generate_n(n,grammar_str=you_need_replace_this_with_name_you_given_2,split_mark='=',line_split_mark='\n',word_split_mark='|'):
#     for i in range(n):
#         a=create_grammar(grammar_str,split_mark, line_split_mark, word_split_mark)
#         sentence=[generate(a,'sentence') for t in random.choice(a[expr])]
    
#     return ','.join(sentence)
# #expanded=[generate(gram,t) for t in choice(gram[target])]
#  #       return ''.join([e if e!='\n' else '\n' for e in expanded if e!='null'])

In [30]:
def generate_n(n,gram,target):
    list1=[]
    for i in range(n):
        list1.append(generate(gram,target)+'！！！')
    return list1

In [31]:
gram_dict=create_grammar(you_need_replace_this_with_name_you_given_2,'=', '\n', '|')
generate_n(3,gram_dict,'sentence')

['你们会不吹捧牛逼的进击的巨人！！！', '哪个会不吹捧巨人！！！', '谁会不吹捧巨人！！！']

>

#### 2. 使用新数据源完成语言模型的训练

按照我们上文中定义的`prob_2`函数，我们更换一个文本数据源，获得新的Language Model:

1. 下载文本数据集（你可以在以下数据集中任选一个，也可以两个都使用）
    + 可选数据集1，保险行业问询对话集： https://github.com/Computing-Intelligence/insuranceqa-corpus-zh/raw/release/corpus/pool/train.txt.gz
    + 可选数据集2：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv
2. 修改代码，获得新的**2-gram**语言模型
    + 进行文本清洗，获得所有的纯文本
    + 将这些文本进行切词
    + 送入之前定义的语言模型中，判断文本的合理程度

In [41]:
FILE2 = open('./movie_comments.csv',encoding='utf-8').read()

In [42]:
FILE2[:500]
#很多标点符号

'id,link,name,comment,star\n1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1\n2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮番上场，视物理逻辑于不顾，不得不说有钱真好，随意胡闹,2\n3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋律，为了煽情而煽情，让人觉得他是个大做作、大谎言家。（7.29更新）片子整体不如湄公河行动，1.整体不够流畅，编剧有毒，台词尴尬；2.刻意做作的主旋律煽情显得如此不合时宜而又多余。,2\n4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4\n5,https://m'

In [43]:
sub_file2 = FILE2[:1000000]

In [46]:
TOKENS2 = cut(sub_file2)

In [71]:
words_count2 = Counter(TOKENS2)
words_count2.most_common(20)
#大部分都是标点符号

[('/', 43329),
 (',', 34550),
 ('.', 18282),
 ('，', 18107),
 (' ', 17524),
 ('的', 14701),
 ('。', 9649),
 (':', 9386),
 ('\n', 8905),
 ('com', 8591),
 ('douban', 8585),
 ('movie', 8584),
 ('https', 8580),
 ('subject', 8578),
 ('了', 5184),
 ('是', 3329),
 ('3', 3012),
 ('4', 2994),
 ('！', 2975),
 ('5', 2522)]

In [72]:
#清洗文字数据
import re
def token(string):
    return re.findall('\w+', string)
#正则表达findall自带把str变成List


In [80]:
TOKENS2=token(FILE2)[:1000000] #findall自带把str变成List
# TOKENS2 = cut(sub_file2)  #不再需要额外切分
words_count2 = Counter(TOKENS2)
words_count2.most_common(20)
#没有很多标点符号了

[('com', 67460),
 ('movie', 67425),
 ('douban', 67390),
 ('https', 67360),
 ('subject', 67359),
 ('4', 20985),
 ('5', 18764),
 ('3', 16193),
 ('2', 9031),
 ('1', 7334),
 ('The', 6881),
 ('of', 2400),
 ('the', 1716),
 ('Man', 1266),
 ('s', 1241),
 ('劇場版', 830),
 ('A', 826),
 ('Me', 795),
 ('You', 594),
 ('to', 562)]

In [48]:
# words_with_fre2= [f for w, f in words_count.most_common()]

In [50]:
# _2_gram_words2 = [
#     TOKENS2[i] + TOKENS2[i+1] for i in range(len(TOKENS2)-1)
# ]
# _2_gram_words2[:10]

['id,',
 ',link',
 'link,',
 ',name',
 'name,',
 ',comment',
 'comment,',
 ',star',
 'star\n',
 '\n1']

In [53]:
# _2_gram_word_counts2 = Counter(_2_gram_words2)

In [54]:
get_gram_count('太恐怖了这个电影',words_count2)

1

In [78]:
two_gram_model('太恐怖了这个电影')

1.7315199560937358e-09

In [79]:
two_gram_model('湄公河行动')

0.002881844380403458

In [57]:
two_gram_model('好看到不像《战狼1》的续集，完虐《湄公河行动')

7.547436953548058e-34

#### 3. 获得最优质的的语言

当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。请定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成**n**个句子，并能选择一个最合理的句子: 



提示，要实现这个函数，你需要Python的sorted函数

In [32]:
sorted([1, 3, 5, 2])

[1, 2, 3, 5]

这个函数接受一个参数key，这个参数接受一个函数作为输入，例如

In [33]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[0])

[(1, 4), (2, 5), (4, 4), (5, 0)]

能够让list按照第0个元素进行排序.

In [34]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[1])

[(5, 0), (1, 4), (4, 4), (2, 5)]

能够让list按照第1个元素进行排序.

In [35]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[1], reverse=True)

[(2, 5), (1, 4), (4, 4), (5, 0)]

能够让list按照第1个元素进行排序, 但是是递减的顺序。

>

In [64]:
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

In [69]:
def generate_best(n,grammar_str,lang_model): # you code here
    gram_dict=create_grammar(grammar_str,'=', '\n', '|')
    lang_list=generate_n(n,gram_dict,'host')
    pr_sentenze=[]
    print('生成的几个句子和概率分别是：\n')
    for sentenze in lang_list:
        pr_sentenze.append((sentenze,lang_model(sentenze)))
        print(sentenze+str(lang_model(sentenze)))
    print('最高概率的句子是：')
    return sorted(pr_sentenze, key=lambda x: x[1], reverse=True)[0][0]

In [70]:
generate_best(10,host,two_gram_model)

生成的几个句子和概率分别是：

小朋友,你好我是8号,请问你要耍一耍打猎吗？！！！3.695326615651304e-41
您好我是73号,请问你要耍一耍打牌吗？！！！4.0217163389787055e-35
先生,您好我是8739号,请问你要玩一玩打猎吗？！！！7.53846629592866e-40
先生,你好我是8号,请问你要耍一耍打牌吗？！！！3.695326615651304e-41
您好我是6号,您需要玩一玩打猎吗？！！！6.157895917908155e-31
女士,您好我是5号,您需要耍一耍打牌吗？！！！2.14921979476395e-38
您好我是3号,您需要耍一耍打牌吗？！！！5.652314431327306e-35
您好我是983号,您需要耍一耍喝酒吗？！！！4.053516920751867e-33
小朋友,你好我是35号,您需要玩一玩打牌吗？！！！1.74518275531647e-36
小朋友,你好我是4号,您需要耍一耍打牌吗？！！！2.743027632231474e-40
最高概率的句子是：


'您好我是6号,您需要玩一玩打猎吗？！！！'

好了，现在我们实现了自己的第一个AI模型，这个模型能够生成比较接近于人类的语言。

>

Q: 这个模型有什么问题？ 你准备如何提升？ 

Ans:
1.生成的句子质量还是比较一般，依赖于配置字符串的合理性，不能跳出条条框框，语言模型忽略了词语之间存在远距离关联的情况。
2.需要人为手动设置，对于超大量句子生成来说不可行。
3.运算速度较慢，10个句子就已经开始要等待了。
提升：
1.利用学习的方法，让句子自己学习配置文件的规则
2.利用更复杂的模型衡量句子生成概率，比如LSTM
3.利用动态规划的方法提升运算效率

>

##### 以下内容为可选部分，对于绝大多数同学，能完成以上的项目已经很优秀了，下边的内容如果你还有精力可以试试，但不是必须的。

#### 4. (Optional) 完成基于Pattern Match的语句问答
> 我们的GitHub仓库中，有一个assignment-01-optional-pattern-match，这个难度较大，感兴趣的同学可以挑战一下。


#### 5. (Optional) 完成阿兰图灵机器智能原始论文的阅读
1. 请阅读阿兰图灵关于机器智能的原始论文：https://github.com/Computing-Intelligence/References/blob/master/AI%20%26%20Machine%20Learning/Computer%20Machinery%20and%20Intelligence.pdf 
2. 并按照GitHub仓库中的论文阅读模板，填写完毕后发送给我: mqgao@kaikeba.com 谢谢

> 

各位同学，我们已经完成了自己的第一个AI模型，大家对人工智能可能已经有了一些感觉，人工智能的核心就是，我们如何设计一个模型、程序，在外部的输入变化的时候，我们的程序不变，依然能够解决问题。人工智能是一个很大的领域，目前大家所熟知的深度学习只是其中一小部分，之后也肯定会有更多的方法提出来，但是大家知道人工智能的目标，就知道了之后进步的方向。

然后，希望大家对AI不要有恐惧感，这个并不难，大家加油！

>

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1561828422005&di=48d19c16afb6acc9180183a6116088ac&imgtype=0&src=http%3A%2F%2Fb-ssl.duitang.com%2Fuploads%2Fitem%2F201807%2F28%2F20180728150843_BECNF.thumb.224_0.jpeg)